# Generating the circuit for N2

## Classical calculations:

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from utility import *
import tequila as tq
threshold = 1e-6 #Cutoff for UCC MP2 amplitudes and QCC ranking gradients

basis = 'sto-3g'

In [ ]:
bond_lengths = np.linspace(0.7,3.0,15)
#Run FCI
FCI_PES = obtain_PES('n2', bond_lengths, basis, method='fci')
#Run HF
HF_PES = obtain_PES('n2', bond_lengths, basis,  method='hf')
#Run CCSD
CCSD_PES = obtain_PES('n2', bond_lengths, basis,  method='ccsd')

E = -105.76498510259827 Eh
E = -107.14429700010217 Eh
E = -107.59093448975125 Eh
E = -107.67740455044407 Eh
E = -107.63992557722375 Eh
E = -107.57281125738827 Eh
E = -107.51301935617232 Eh
E = -107.47386970011628 Eh
E = -107.45399643946577 Eh
E = -107.4455026786547 Eh
E = -107.44200151116885 Eh
E = -107.44036442101 Eh
E = -107.43943657892184 Eh
E = -107.4388541693817 Eh
E = -107.4384908527115 Eh
E = -105.69764181746334 Eh
E = -107.04629760546 Eh
E = -107.45386359104577 Eh
E = -107.49049231337895 Eh
E = -107.3923481975834 Eh
E = -107.25392034947382 Eh
E = -107.11129982734893 Eh
E = -106.97876183913667 Eh
E = -106.86232997228902 Eh
E = -106.76366889102728 Eh
E = -106.68178308525789 Eh
E = -106.61455681248016 Eh
E = -106.55979437594225 Eh
E = -106.51555388907731 Eh
E = -106.48009667128879 Eh
E = -105.76423723955378 Eh
E = -107.1428247528051 Eh
E = -107.5880262558067 Eh
E = -107.67166862083374 Eh
E = -107.62967665326595 Eh
E = -107.55886471578982 Eh
E = -107.5072618088078 Eh
E = -107.51164

In [ ]:
#Plot LiH PESs

plt.title('N2 dissociation, STO-3G')
plt.xlabel('R, Angstrom')
plt.ylabel('E, Hartree')

plt.plot(bond_lengths, FCI_PES, label='FCI')
plt.scatter(bond_lengths, HF_PES, label='HF', color='orange')
plt.scatter(bond_lengths, CCSD_PES, label='CCSD', color='purple')
plt.legend()

## Generating Qubit Hamiltonians

In [ ]:
qubit_transf = 'jw' # Jordan-Wigner transformations
n2 = get_qubit_hamiltonian(mol='n2', geometry=1, basis='sto3g', qubit_transf=qubit_transf)
print(n2)
#Unsure of geometry of N2 -- TODO

In [ ]:
n2_tapered = taper_hamiltonian(n2, n_spin_orbitals=12, n_electrons=4, qubit_transf=qubit_transf)
#Unsure of spin orbitals and electrons --- TODO

## Tequila stuff

In [ ]:
trotter_steps = 1

xyz_data = get_molecular_data('n2', geometry=1.5, xyz_format=True)
basis='sto-3g'
#Unsure of geometry of N2 -- TODO

n2_tq = tq.quantumchemistry.Molecule(geometry=xyz_data, basis_set=basis)

print('Number of spin-orbitals (qubits): {} \n'.format(2*n2_tq.n_orbitals))

E_FCI = n2_tq.compute_energy(method='fci')

print('FCI energy: {}'.format(E_FCI))

In [ ]:
H = n2_tq.make_hamiltonian()

print("\nHamiltonian has {} terms\n".format(len(H)))

U_UCCSD = n2_tq.make_uccsd_ansatz(initial_amplitudes='MP2',threshold=threshold, trotter_steps=trotter_steps)

E = tq.ExpectationValue(H=H, U=U_UCCSD)

print('\nNumber of UCCSD amplitudes: {} \n'.format(len(E.extract_variables())))

print('\nStarting optimization:\n')

result = tq.minimize(objective=E, method="BFGS", initial_values={k:0.0 for k in E.extract_variables()}, tol=1e-6)

print('\nObtained UCCSD energy: {}'.format(result.energy))

In [ ]:
#Define number of entanglers to enter ansatz
n_ents = 1

#Rank entanglers using energy gradient criterion
ranked_entangler_groupings = generate_QCC_gradient_groupings(H.to_openfermion(), 
                                                             2*n2.n_orbitals, 
                                                             hf_reference, 
                                                             cutoff=threshold)

print('Grouping gradient magnitudes (Grouping : Gradient magnitude):')
for i in range(len(ranked_entangler_groupings)):
    print('{} : {}'.format(i+1,ranked_entangler_groupings[i][1]))


entanglers = get_QCC_entanglers(ranked_entangler_groupings, n_ents, 2*n2.n_orbitals)

print('\nSelected entanglers:')
for ent in entanglers:
    print(ent)

In [ ]:
H = tq.QubitHamiltonian.from_openfermion(get_qubit_hamiltonian('n2', 1.5, 'sto-3g', qubit_transf='jw'))
#Unsure of geometry of N2 -- TODO

a = tq.Variable("tau_0")
U = construct_QMF_ansatz(8)
for ent in entanglers:
    U += tq.gates.ExpPauli(paulistring=tq.PauliString.from_string(ent), angle=a)
print(U)
